In [1]:
# Подключаем библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tabula as tb
from scipy import stats
plt.rcParams['figure.figsize']=(10,5)